# Talktorial 12

# Binding Site Detection (DogSiteScorer)

Abishek Laxmanan Ravi Shankar

## Aim of the talktorial

To get a protein structure and **detect its ligand binding** site at the example of **EGFR**.

Be more precise (can be done once you outlined the practical).

# Learning goals

### Theory
* Protein binding sites
* Binding site detection 
    * Introduction
    * DogSiteScorer    
* Validation
    * Ligand-based detection
    * KLIFS pocket definitions

### Practical

TBA

## References

* Combining global and local measures for structure-based druggability predictions.
([<i>Noordwijkerhout Cheminformatics</i> 2011, <b>52</b>, 360-372](https://www.ncbi.nlm.nih.gov/pubmed/22148551))

* Prediction, Analysis, and Comparison of Active Sites 
([ <i>Journal of Chemical Information and Modeling</i> 2018 <b>6,7</b>](https://onlinelibrary.wiley.com/doi/abs/10.1002/9783527806539.ch6g))

* DoGSiteScorer
([<i>Bioinformatics</i> 2012, <b>28</b>, 2074-2075](https://doi.org/10.1093/bioinformatics/bts310)) 

* Fpocket
([<i>BMC Bioinformatics</i> 2009, <b>10</b>, 168](https://bmcbioinformatics.biomedcentral.com/articles/10.1186/1471-2105-10-168))

* Protein–ligand complex
([Protein comlex wiki](https://en.wikipedia.org/wiki/Protein%E2%80%93ligand_complex))

* Voronoi tessellation
([Voronoi tessellation Wiki](https://en.wikipedia.org/wiki/Voronoi_diagram))

* KLIFS database
( [KLIFS Website](http://klifs.vu-compmedchem.nl/))  

* Fragments based drug Discovery
([Google books](https://books.google.de/books?id=tjfvCQAAQBAJ&pg=PA202&lpg=PA202&dq=Fragments+based+drug+Discovery+SURFNET&source=bl&ots=O6XVGMVPTV&sig=ACfU3U2K6btUeMpgN4FBe627jpIJL6-mtQ&hl=en&sa=X&ved=2ahUKEwjonezYmbLkAhXBRxUIHdzKClIQ6AEwAnoECAkQAQ))

* KLIFS: a structural kinase-ligand interaction database
([<i>KLIFS Pubmed</i> 2016, Nucleic Acids Research, <b>44</b>, 365-371](https://www.ncbi.nlm.nih.gov/pubmed/26496949)) 

* KLIFS: A Knowledge-Based Structural Database To Navigate Kinase−Ligand Interaction Space
([<i>KLIFS Journal of Medicinal Chemistry</i> 2013, <b>57</b>, 249-257](https://pubs.acs.org/doi/10.1021/jm400378w))

* SURFNET
([<i>Journal of Molecular Graphics</i>1995, <b>13</b>323-330](https://www.ncbi.nlm.nih.gov/pubmed/8603061))

* LIGSITE
([<i>Journal of Molecular Graphics and Modelling</i> 1997 <b>6</b> 359-363](https://www.sciencedirect.com/science/article/pii/S1093326398000023?via%3Dihub))

* Docking 
([Docking wiki](https://en.wikipedia.org/wiki/Docking_(molecular)))


## Theory

### Protein binding sites

The main objective of a pharmaceutical research is to find a new drug inorder to cure a specific disease. Structure-based approaches in drug design develop drugs with respect to a target protein which is associated with the disease under investigation. However, in order to develop such a drug it is crucial to know where the drug is supposed to bind in the protein and therefore inhibit or activate the protein. Not always such a small molecule binding site is known. Therefore it is important to have tools for binding site detection.

Binding site detection has a special mention in the process of drug development. Binding sites are defined as a **hollow 3 dimensional space** on the surface of a protein molecule, which serves as the region for **ligand**, **peptide**, or **protein** binding. There are various factors which influence whether such a region is suitable binding site.
* The Shape
* The Spatial arrangement of the residues
* The Possible interactions are
    * Van der Waals interactions
    * Hydrogen bond interactions
    * Hydrophobic regions
    * Electrostatic interactions
    * π-π interactions
   


### Binding site detection 

If ligand information is available (protein-ligand-complex), then the ligand-surrounding protein region is defined as pocket (e.g. using a 6 Å radius).
If the ligand is absent, then the detection tools can be used in the process of pocket detection. This comprises of 2 standard approaches. These methods can be grouped into,**Geometry-based and Energy-based** approaches as well as **Grid-based and Grid-free** approaches as outlined in the Figure 1.

<div style="width:500; font-size:100%; text-align:center;">
    <img src="images/Figure1_DetectionMethods.png" alt="alternate text"  width="500" style="padding:0.8em;"/>
    Figure 1: Binding Site detection methods can be grouped into geometry-based and energy-based approaches as well as grid-based and grid-free approaches .
</div>

#### Geometry-based approaches

Geometry-based approaches analyze the shape of a molecular surface to locate cavities. It is based upon the 3D spatial arrangement of the atoms on the protein surface. They are broadly divided into two namely Grid-Based and Grid-free approaches.

* **Grid-based approaches**

    * Examples: POCKET, LIGSITE and DoGSiteScorer 
    * In LIGSITE, the cartesian coordinates runs along the X, Y and Z axes through a 3D region which consists of protein surface on both sides.
    * In addition to these axes a cubic diagonal passes along to find out the width of the cleft in between the protein surface. Therefore LIGSITE checks in seven directions the distance of each atom in the protein surface.
    * Therefore these coordinates enables LIGSITE to diffrentate the Protein Solvent Protein regions (PSP) on the protein surface
    * DoGSiteScorer is explained in detail below.
   
* **Grid-free approaches**

    * Example: SURFNET
    * In SURFNET spheres are placed on the protein surface instead of having grids. 
    * The probe spheres are placed midway between the pairs of atoms on the protein. Incase a probe overlaps any nearby atom, its radius is reduced until no overlap occurs. 
    * The resulting probes defines the pocket and the cavities. The cutoff radius of the spheres are only with a radius of 1 to 4 angstrom are kept.

#### Energy-based approaches

Interaction of a molecular fragment with protein is recorded. Favourable energitic responses are assigned to pockets. DrugSite and Docking are the well known energy based algorithms. They are again divided into grid-based and grid-free approaches.
<div class="center" align="right" width="500"> </div>

* **Grid-based approaches**

    * Eg: Drugsite
    * This method uses carbon probes placed on each grid point, and van der Wall's energies between the probes within a distance of 8 Å are calculated. Mean energy and Standard deviation are calculated for these grid points. 
    * Grid points with unfavourable energies are removed. Grid points fulfilling these cut-off are merged to pockets.
   
* **Grid-free approaches**

    * Eg: Docking
    * Docking process is similar to "lock and key" model in which the **Target protein resembles** the **lock** and **ligand** to the **key**
    * In this method, a scoring function is incorporated to evaluate the fragments that are placed at a position on the protein surface.
    * Scoring functions are physics based, which uses Molecular Mechanics force fields, which primarily estimates the amount of affinity between the binding ligand and the protein.
    * The scoring functions calculates the feasibilty of whether the ligand could be docked to the particular protein
    * These pockets are assigned based on the quantity of fragments that bind to a specific area.

### DogSiteScorer 

Certain geometric and physico-chemical properties are calculated in an automatic manner for the predicted pockets and subpockets. In **DogSiteScorer** , pockets and subpockets are predicted with DoGSite using a Difference of **Gaussian filter**. DogSiteScorer falls under the category of the grid based approaches. Pocket volume and surface are calculated by counting the **grid points** constituting the pocket volume or its surface and multiplying this number with the grid box volume or surface, respectively. A **breadth-first search** is used for pocket depth computation, starting from the solvent exposed pocket parts toward the most deeply buried regions. 

#### Pocket Detection

* The protein structure is covered around with a grid of coordinates.
* Each of the **grid points** are **labelled**.
* Difference of **Gaussian filter** is applied across the grid.
* This operation helps to find the position on a protein surface where the location of a **sphere-like object** is favorable.
* The last step in this algorithm is the merging of neighbouring subpockets to the pockets.

<div style="width:400; font-size:100%; text-align:center;">
    <img src="images/Figure2_BindingSitePocket.png" alt="alternate text"  width="400" style="padding:0.8em;"/>
    Figure 2: The binding site on the surface of the protein shown with grid view consisting of the surface molecules, volume inside the binding region, and the region exposed to the exterior of the protein 
</div>

#### Druggability 

* The Druggability is based on Machine learning technique - **Support Vector Machine (SVM)**.
* **Discriminative Analysis** - Best suited to seperate druggable from undruggable.
* The model has been trained and tested on both the redundant and non-redundant version of the druggable dataset
* This model showed a mean accuracy of 90%.
* The druggabilty score lies between 0 and 1. The higher the score the better the chance that the pocket is druggable.
    
<div style="width:1000; font-size:100%; text-align:center;">
    <img src="images/Figure3_DogSiteAlgo.png" alt="alternate text"  width="1000" style="padding:0.8em;"/>
    Figure 3: Flowsheet explaining the pipeline of DoGSiteScorer
</div>


### Validation methods

#### Ligand-based detection

Sometimes ligand information is available (protein-ligand-complex), in that case usually the ligand-surrounding protein region is defined as pocket (e.g. using a 6 Å radius).

If a bound ligand is known for a structure, it can be used for binding-site annotation.  


#### KLIFS pocket definition

KLIFS stands for **Kinase-Ligand Interaction Fingerprints and structures database**. It is a structural repository of over 2900 human and mouse kinase enzymes along with their catalytic activity. It also deals with the kinase domains and also the inhibitors which can interact with them. Kinases share a similar conserved regions, which serves as a challenge for small molecules that can selectively bind to a specific kinase. 

KLIFS enables us to do systematic comparison and analysis of chemical features of all available kinases in the process of ligand binding. The classification helps us of an all-encompassing binding site of 85 residues. It is possible to compare the **interaction patterns** of **kinase-inhibitors** to each other to, for example, identify crucial interactions determining kinase-inhibitor selectivity. Therefore this could be an ideal method used for the validation of the obtained binding site. 

## Practical



In [36]:
import requests

import pandas as pd

In [37]:
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

In [38]:
def dogsite_scorer_submit_with_pdbid(pdb_code, chain_id, ligand=''):
    """This is the official API, but they only allow PDB codes, not custom ones..."""
    # Submit job to proteins.plus
    r = requests.post("https://proteins.plus/api/dogsite_rest",
        json={
            "dogsite": {
                "pdbCode": pdb_code,
                "analysisDetail": "1",
                "bindingSitePredictionGranularity": "1",
                "ligand": ligand,
                "chain": chain_id
            }
        },
        headers= {'Content-type': 'application/json', 'Accept': 'application/json'}
    )

    r.raise_for_status()
    # We have to query location for updates on the calculation
    return r.json()['location']

In [39]:
def dogsite_scorer_guess_binding_site(protein):
    """
    Use proteins.plus' DoGSiteScorer to retrieve most probable binding site in protein.
    """
    if len(protein) == 4:  # pdb code
        job_location = dogsite_scorer_submit_with_pdbid(protein)
    elif protein.endswith('.pdb'):
        job_location = dogsite_scorer_submit_with_custom_pdb(protein)
    else:
        raise ValueError("`protein` must be a PDB ID or a path to a .pdb file!")
    
    # Check when the calculation has finished
    while True:
        result = requests.get(job_location)
        result.raise_for_status()  # if it fails, it will stop here
        if result.status_code == 202:  # still running
            time.sleep(5)
            continue
        break
    
    # the residues files contain the geometric center and radius as a comment in the PDB file
    # first file (residues[0]) is the best scored pocket
    pdb_residues = requests.get(result.json()['residues'][0]).text
    for line in pdb_residues.splitlines():
        line = line.strip()
        if line.startswith('HEADER') and 'Geometric pocket center at' in line:
            fields = line.split()
            center = [float(x) for x in fields[5:8]]
            radius = float(fields[-1])
            break
    return center, radius  # this is what we need for our Vina calculation

In [40]:
job_location = dogsite_scorer_submit_with_pdbid('3w32', 'A')
job_location

'https://proteins.plus/api/dogsite_rest/Yn2WyiSZp5wPh6oF6TjnKxag'

In [41]:
result = requests.get(job_location)
result

<Response [200]>

In [43]:
result.json()
result.json().keys()
result.json()['result_table']
result_table = requests.get(result.json()['result_table']).text
result_table
# Split string into list of lists (=table)
result_table_split = [i.split('\t') for i in result_table[:-1].split('\n')]
result_table_split
# Remove spaces
result_table_split = [[j.replace(' ', '') for j in i] for i in result_table_split]
result_table_split


[['name',
  'lig_cov',
  'poc_cov',
  'lig_name',
  'volume',
  'enclosure',
  'surface',
  'depth',
  'surf/vol',
  'lid/hull',
  'ellVol',
  'ellc/a',
  'ellb/a',
  'siteAtms',
  'accept',
  'donor',
  'hydrophobic_interactions',
  'hydrophobicity',
  'metal',
  'Cs',
  'Ns',
  'Os',
  'Ss',
  'Xs',
  'negAA',
  'posAA',
  'polarAA',
  'apolarAA',
  'ALA',
  'ARG',
  'ASN',
  'ASP',
  'CYS',
  'GLN',
  'GLU',
  'GLY',
  'HIS',
  'ILE',
  'LEU',
  'LYS',
  'MET',
  'PHE',
  'PRO',
  'SER',
  'THR',
  'TRP',
  'TYR',
  'VAL',
  'simpleScore',
  'drugScore'],
 ['P_0',
  '0.00',
  '0.00',
  '""',
  '1422.66',
  '0.10',
  '1673.75',
  '19.26',
  '1.176493329397045',
  '-',
  '-',
  '0.13',
  '0.67',
  '288',
  '86',
  '40',
  '71',
  '0.36',
  '0',
  '198',
  '45',
  '41',
  '4',
  '0',
  '0.10',
  '0.13',
  '0.24',
  '0.53',
  '4',
  '5',
  '2',
  '5',
  '2',
  '2',
  '1',
  '5',
  '0',
  '3',
  '12',
  '3',
  '2',
  '3',
  '3',
  '1',
  '2',
  '1',
  '1',
  '5',
  '0.63',
  '0.810023'],

In [44]:
# Extract column names, index names, table body
column_names = result_table_split[0]
index_names = [i[0] for i in result_table_split[1:]]
table = [i[1:] for i in result_table_split[1:]]
table

[['0.00',
  '0.00',
  '""',
  '1422.66',
  '0.10',
  '1673.75',
  '19.26',
  '1.176493329397045',
  '-',
  '-',
  '0.13',
  '0.67',
  '288',
  '86',
  '40',
  '71',
  '0.36',
  '0',
  '198',
  '45',
  '41',
  '4',
  '0',
  '0.10',
  '0.13',
  '0.24',
  '0.53',
  '4',
  '5',
  '2',
  '5',
  '2',
  '2',
  '1',
  '5',
  '0',
  '3',
  '12',
  '3',
  '2',
  '3',
  '3',
  '1',
  '2',
  '1',
  '1',
  '5',
  '0.63',
  '0.810023'],
 ['0.00',
  '0.00',
  '""',
  '599.23',
  '0.06',
  '540.06',
  '17.51',
  '0.9012566126529045',
  '-',
  '-',
  '0.14',
  '0.22',
  '131',
  '35',
  '13',
  '25',
  '0.34',
  '0',
  '95',
  '16',
  '17',
  '3',
  '0',
  '0.03',
  '0.10',
  '0.28',
  '0.59',
  '1',
  '2',
  '1',
  '1',
  '2',
  '1',
  '0',
  '2',
  '0',
  '2',
  '7',
  '1',
  '2',
  '2',
  '1',
  '0',
  '2',
  '0',
  '0',
  '2',
  '0.59',
  '0.620201'],
 ['0.00',
  '0.00',
  '""',
  '201.73',
  '0.08',
  '381.07',
  '11.36',
  '1.8890100629554356',
  '-',
  '-',
  '0.17',
  '0.25',
  '51',
  '17',
  

In [45]:
result_table_df = pd.DataFrame(
    table,
    columns=column_names[1:],
    index=index_names
)
result_table_df.index.name = 'name'
result_table_df

,lig_cov,poc_cov,lig_name,volume,enclosure,surface,depth,surf/vol,lid/hull,ellVol,ellc/a,ellb/a,siteAtms,accept,donor,hydrophobic_interactions,hydrophobicity,metal,Cs,Ns,Os,Ss,Xs,negAA,posAA,polarAA,apolarAA,ALA,ARG,ASN,ASP,CYS,GLN,GLU,GLY,HIS,ILE,LEU,LYS,MET,PHE,PRO,SER,THR,TRP,TYR,VAL,simpleScore,drugScore
name,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
P_0,0.00,0.00,"""""",1422.66,0.10,1673.75,19.26,1.176493329397045,-,-,0.13,0.67,288,86,40,71,0.36,0,198,45,41,4,0,0.10,0.13,0.24,0.53,4,5,2,5,2,2,1,5,0,3,12,3,2,3,3,1,2,1,1,5,0.63,0.810023
P_0_0,0.00,0.00,"""""",599.23,0.06,540.06,17.51,0.9012566126529045,-,-,0.14,0.22,131,35,13,25,0.34,0,95,16,17,3,0,0.03,0.10,0.28,0.59,1,2,1,1,2,1,0,2,0,2,7,1,2,2,1,0,2,0,0,2,0.59,0.620201
P_0_1,0.00,0.00,"""""",201.73,0.08,381.07,11.36,1.8890100629554356,-,-,0.17,0.25,51,17,9,10,0.28,0,36,6,7,2,0,0.08,0.17,0.25,0.50,1,1,0,1,1,1,0,0,0,0,3,1,1,0,0,0,1,0,0,1,0.17,0.174816
P_0_2,0.00,0.00,"""""",185.60,0.17,282.00,9.35,1.519396551724138,-,-,0.45,0.55,48,17,8,12,0.32,0,31,8,8,1,0,0.17,0.25,0.08,0.50,0,2,0,1,0,0,1,1,0,0,2,1,1,1,0,0,0,0,0,2,0.13,0.195695
P_0_3,0.00,0.00,"""""",175.30,0.15,297.42,9.29,1.6966343411294922,-,-,0.23,0.37,48,16,8,14,0.37,0,32,8,8,0,0,0.14,0.14,0.36,0.36,1,1,1,2,0,0,0,3,0,0,1,1,0,1,1,1,0,0,0,1,0.13,0.168845
P_0_4,0.00,0.00,"""""",170.37,0.08,390.10,11.99,2.2897223689616717,-,-,0.16,0.20,47,14,7,17,0.45,0,34,7,6,0,0,0.00,0.18,0.18,0.64,2,2,0,0,0,1,0,0,0,1,3,0,0,0,0,0,0,1,1,0,0.15,0.223742
P_0_5,0.00,0.00,"""""",90.43,0.24,177.50,6.24,1.9628441888753732,-,-,0.70,0.89,26,8,6,5,0.26,0,16,6,4,0,0,0.12,0.25,0.25,0.38,0,1,1,1,1,0,0,0,0,0,0,1,0,0,1,0,0,0,0,2,0.00,0.165232
P_1,0.00,0.00,"""""",708.99,0.13,1030.19,14.32,1.4530388298847656,-,-,0.14,0.59,140,44,13,34,0.37,0,98,17,25,0,0,0.14,0.11,0.36,0.39,3,1,1,0,0,0,4,4,0,1,4,2,0,1,1,2,2,0,1,1,0.46,0.755915
P_1_0,0.00,0.00,"""""",496.90,0.11,739.17,12.72,1.4875628899174884,-,-,0.14,0.18,103,34,8,22,0.34,0,74,12,17,0,0,0.18,0.09,0.32,0.41,2,1,1,0,0,0,4,1,0,1,4,1,0,0,1,2,2,0,1,1,0.49,0.465489


In [ ]:
pdb_residues = requests.get(result.json()['residues'][0]).text
pdb_residues

In [29]:
job_location = dogsite_scorer_submit_with_pdbid('3w32', 'A')
result = requests.get(job_location)
result.json()
result.json().keys()
result.json()['result_table']
result_table = requests.get(result.json()['result_table']).text
# Split string into list of lists (=table)
result_table_split = [i.split('\t') for i in result_table[:-1].split('\n')]
# Remove spaces
result_table_split = [[j.replace(' ', '') for j in i] for i in result_table_split]
# Extract column names, index names, table body
column_names = result_table_split[0]
index_names = [i[0] for i in result_table_split[1:]]
table = [i[1:] for i in result_table_split[1:]]
result_table_df = pd.DataFrame(
    table,
    columns=column_names[1:],
    index=index_names
)
result_table_df.index.name = 'name'
pdb_residues = requests.get(result.json()['residues'][0]).text
pdb_residues

'HEADER\tOutput of DoGSiteScorer by A. Volkamer\nHEADER\tPocket 0 with 288 binding site atoms written.\nHEADER\tReferences: \nHEADER\tA. Volkamer et al. Analyzing the topology of active sites: on the prediction of pockets and subpockets. J. Chem. Inf. Model. 2010,50(11), 2041-52\nHEADER\tA. Volkamer et al. Combining global and local measures for structure-based druggability predictions. J. Chem. Inf. Model. 2012,52,360-372\nHEADER\tGeometric pocket center at  13.65  29.95   8.29 with max radius 20.00\nATOM      2  CA  GLN A 701      -0.291  31.978  -3.835  0.00  0.00           C\nATOM      3  C   GLN A 701       0.946  31.062  -3.957  0.00  0.00           C\nATOM      4  O   GLN A 701       0.876  29.863  -3.659  0.00  0.00           O\nATOM      6  CG  GLN A 701      -2.441  30.619  -3.562  0.00  0.00           C\nATOM     12  C   ALA A 702       3.775  30.322  -3.161  0.00  0.00           C\nATOM     13  O   ALA A 702       3.538  30.924  -2.110  0.00  0.00           O\nATOM     15  

In [30]:
def step_01(pdb_code, chain, alternate_model):
    '''
    Get pocket data.
    '''
    
    # Get pocket from DoGSiteScorer
    pockets_dogsite = get_dogsite_pockets(
        pdb_code, 
        analysis_detail, 
        binding_site_prediction_granularity, 
        ligand='', 
        chain=''
    )
    pocket_dogsite = select_dogsite_pocket(pockets_dogsite)
    
    # Get pocket from KLIFS
    pocket_klifs = get_klifs_pocket(
        pdb_code, 
        chain='', 
        alternate_model=''
    )
    
    # Get pocket based on ligand
    pocket_ligand = get_pocket_from_ligand(
        pdb_code,
        ligand_code, 
        chain='', 
        alternate_model='', 
        radius=6.5
    )
    
    return pocket_dogsite, pocket_klifs, pocket_ligand

## Appendix

#### Introduction to the used methods
 
 There are 3 automatic methods for the pocket and the druggability predictions. They include,
 * Sitemap (2009)
 * Fpocket (2009/2010)
 * DogSiteScorer (2010)
 
 All 3 algorithms predict pockets solely based on the **atomic co-ordinates** of the protein.
 
 * **SiteMap** uses **van der Waals (VdW) energies** and a **buriedness value** calculated on a grid to predict pockets on the protein surface.
 * **Fpocket** investigates **α spheres** for active site predictions.
 * In **DoGsiteScorer** , pockets and subpockets are predicted with DoGSite using a Difference of **Gaussian filter**.

***The shape (and the molecular orientation) of the region

 * (London dispersion forces)
    * (Covalent interaction)
  
  align="right" width="260"

Ref

* D. Durrant et al., 2009 **POVME: An Algorithm for Measuring Binding-Pocket Volumes**


If we have a ligand: Use ligand radius

If we don't have a ligand: Detection tools

* Grid-based vs. grid-free

<div class="center" align="right" width="200">Figure 2: Energy based and geometry <br>based methods </div>

* The radii is scaled down to ensure that they don't clash with the surrounding atom of a protein.
    * Small virtual probes are used to cover and coat the protein surface.  Number of probes surrounding each atom is calculated. The spaces on the outer surfaces are filled and the potential binding site centers are selected.

* 
 
#### Fpocket 

Fpocket stands for "Finding Pocket". The pocket detection package based on Voronoi tessellation and alpha spheres. **Fpocket** investigates **α spheres** for active site predictions. Fpocket algorithm consists of three major steps.

* <i>Step 1:</i> The whole ensemble of alpha spheres is determined from the protein structure. Fpocket returns a   pre-filtered collection of spheres.
* <i>Step 2:</i> Identifying the clusters of spheres close together, to identify pockets, and to remove clusters of poor interest.
* <i>Step 3:</i> This finally calculates properties from the atoms of the pocket, in order to score each pocket.

<div style="width:400; font-size:90%; text-align:center;">
    <img src="images/Figure4_VoronoiTess.png" alt="alternate text"  width="400" style="padding:0.8em;"/>
    Figure 4: Voronoi tessellation based method
</div>



Grids with cartesian co-ordinates are used to embed a protein with spacing between 0.4 Å and 2.0 Å
    * *This is assigned based upon the Van der Waal's radii* surrounding an atom of a protein.  
    
    
    * "*SURFNET is a common algorithm for the grid-free methods.
    
    * *DrugSite is a grid-based approach under the class of Energy based algorithms.
    
    
    * Docking serves as an important method in the Energy-based grid-free approach.

In [31]:
def get_dogsite_pockets(pdb_code, analysis_detail, binding_site_prediction_granularity, ligand='', chain=''):
    pass

def select_dogsite_pocket():
    pass

In [32]:
def get_klifs_pocket(pdb_code, chain='', alternate_model=''):
    pass

In [33]:
def get_pocket_from_ligand(pdb_code, ligand_code, chain='', alternate_model='', radius=6.5):
    pass

In [34]:
def step_02(pocket_list):
    '''
    Compare different pocket results.
    '''
    
    pass

In [35]:
def step_03():
    '''
    Visualize different pocket results.
    # TODO:
    - NGLViewer
    '''
    
    pass